# Just trash notebook, I use it as python repl alt

In [2]:
import numpy as np

import torch
from torch import nn
from torch import optim

In [3]:
input_size = 26
hidden_size = 128
num_layers = 3
batch_first = True
batch_size = 128
bidirectional = False

model = nn.RNN(
    input_size=input_size,
    hidden_size=hidden_size,
    num_layers=num_layers,
    batch_first=batch_first,
    bidirectional=bidirectional,
)

sequence_length = 5
x = torch.randn(
    batch_size,
    sequence_length,
    input_size,
)

h0 = torch.randn(
    num_layers,
    batch_size,
    hidden_size,
)

y = torch.randn(
    batch_size,
    sequence_length,
    hidden_size,
)

loss_fn = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=1e-3)


pred, hn = model(x, h0)
loss = loss_fn(y, pred)

optimizer.zero_grad()
loss.backward()
optimizer.step()

print(loss.item())

1.1026149988174438


In [28]:
rnn_layer = nn.RNN(input_size=5, hidden_size=2, num_layers=1, batch_first=True)

w_xh = rnn_layer.weight_ih_l0
w_hh = rnn_layer.weight_hh_l0
b_xh = rnn_layer.bias_ih_l0
b_hh = rnn_layer.bias_hh_l0

print("W_xh", rnn_layer.weight_ih_l0.shape)
print("W_hh", rnn_layer.weight_hh_l0.shape)
print("b_xh", rnn_layer.bias_ih_l0.shape)
print("b_hh", rnn_layer.bias_hh_l0.shape)

W_xh torch.Size([2, 5])
W_hh torch.Size([2, 2])
b_xh torch.Size([2])
b_hh torch.Size([2])


In [42]:
x_seq = torch.tensor([[1.0] * 5, [2.0] * 5, [3.0] * 5]).float()  # shape (3, 5)
x_seq_with_batch = torch.reshape(x_seq, (1, 3, 5)) 
# shape (1, 3, 5); we added batch shape
# 1 - batch
# 3 - sequnce length
# 5 - features
# Imagine x sequnce is input for text generation model
# then 1-batch     is number of text chunks -> truncated sentences?
#      3-seq       is sentence length
#      5-features  is vocabulary size (one-hot encoding of character)

output, hn = rnn_layer(x_seq_with_batch)

out_manually = []
for t in range(3): # 3 sentences
    xt = torch.reshape(x_seq[t], (1, 5))  # (5,) => (1, 5)
    print(f"Time step {t} =>")
    print('    Input             :', xt.numpy())
    ht = xt @ w_xh.T + b_xh
    print('    Hidden            :', ht.detach().numpy())

    if t == 0:
        prev_h = torch.zeros((ht.shape))
    else:
        prev_h = out_manually[t-1]
    
    ot = ht + prev_h @ w_hh.T + b_hh
    ot = torch.tanh(ot)
    out_manually.append(ot)
    print('    Output (manual)   :', ot.detach().numpy())
    print('    RNN output        :', output[:, t].detach().numpy())
    print()
    

Time step 0 =>
    Input             : [[1. 1. 1. 1. 1.]]
    Hidden            : [[-0.34027523  0.5212964 ]]
    Output (manual)   : [[-0.6049818   0.04433013]]
    RNN output        : [[-0.6049818   0.04433013]]

Time step 1 =>
    Input             : [[2. 2. 2. 2. 2.]]
    Hidden            : [[-0.00453627  0.7305054 ]]
    Output (manual)   : [[-0.48325324 -0.07878461]]
    RNN output        : [[-0.48325324 -0.07878458]]

Time step 2 =>
    Input             : [[3. 3. 3. 3. 3.]]
    Hidden            : [[0.33120275 0.9397144 ]]
    Output (manual)   : [[-0.2117602  0.1303394]]
    RNN output        : [[-0.21176018  0.1303394 ]]

